In [1]:
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import string
import gensim

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'll": "i will",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "needn't": "need not",
    "oughtn't": "ought not",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that'd": "that would",
    "that's": "that is",
    "there'd": "there had",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where'd": "where did",
    "where's": "where is",
    "who'll": "who will",
    "who's": "who is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are"
}

stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stemmer = None
# lemmatizer = None

TEST_SIZE = 0.1
SMOOTHING = 1.0
NGRAM_RANGE = (1, 1)

## Helper Functions

In [3]:
def preprocess(sentence, lower_case=True, remove_punctuation=True, replace_contractions=True):
    if lower_case:
        sentence = sentence.lower()
    if remove_punctuation:
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    if replace_contractions:
        news = sentence.split()
        new_news = []
        for word in news:
            if word in contractions:
                new_news.append(contractions[word])
            else:
                new_news.append(word)
        sentence = " ".join(new_news)
    return sentence

In [4]:
def tokenize(sentence, stemmer=stemmer, lemmatizer=lemmatizer, remove_stop_words=False):
    tokens = word_tokenize(sentence)
    
    if remove_stop_words:
        tokens = [token for token in tokens if token not in stop_words]
    if stemmer:
        tokens = [stemmer.stem(token) for token in tokens]
    if lemmatizer:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
    return tokens

## Load Data

In [5]:
full_train_df = pd.read_csv('raw_data/fulltrain.csv', header=None)
full_train_df.columns = ['label', 'text']
full_train_df.head()

,label,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [6]:
train_df = full_train_df.drop_duplicates(subset=['text'])
print(f"No. training samples (all classes): {len(train_df)}")

No. training samples (all classes): 48652


## Word2Vec

In [7]:
train_df['text_clean'] = train_df['text'].apply(lambda x: preprocess(x))
train_df['clean_tokens'] = train_df['text_clean'].apply(lambda x: tokenize(x))
X = train_df['clean_tokens'].values
y = train_df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, stratify=y, random_state=42)
X_train.shape, X_test.shape

((43786,), (4866,))

In [8]:
# Train the word2vec model 
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=300,
                                   window=10)
words = set(w2v_model.wv.index_to_key)
print(words)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train])

{'daylight', 'twittering', 'tecmo', 'dismemberment', 'herring', 'chairmanship', 'maclean', 'sdf', '42', 'dfl', 'refinanced', 'failureby', 'pancreatic', 'heinz', 'wittman', 'coffer', 'earplug', 'struggle', 'nonstarter', 'wil', 'vejjajiva', 'malishevski', 'quickthinking', 'slime', 'craigs', 'paramount', 'ofus', '060', 'peril', 'studious', 'tinder', 'wizard', 'influencepeddling', 'battelle', 'waived', 'plumber', 'alexandra', 'abbreviated', 'natural', 'tbtf', 'natwest', 'mcchrystals', 'acai', 'disa', 'oafish', 'ohiobased', 'guardsman', 'sugarcoated', 'willow', 'padlock', 'utters', 'twobyfour', 'capitulation', 'a21', 'kfor', 'wildcard', 'checkmate', '1511', 'excursion', 'messiah', 'palestinian', 'bse', 'await', 'teamster', 'kupchak', 'vactruth', '18111812', 'mcnally', 'herrin', 'vatgrown', 'blackwill', 'gobal', 'contributed', 'blevins', 'nursed', 'dracula', 'gertrudes', 'designate', '760', 'nannystate', 'browders', 'kijera', 'flagwaving', 'anybody', '3006', 'orb', 'favre', '528', 'anticompe

In [9]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(300, dtype=float))

In [10]:
clf = LogisticRegression(random_state=0, max_iter=200, solver='liblinear').fit(X_train_vect_avg, y_train)
y_pred = clf.predict(X_train_vect_avg)
print(f"Accuracy: {accuracy_score(y_train, y_pred)}")
print(f"F1 score: {f1_score(y_train, y_pred, average='macro')}")

Accuracy: 0.9397752706344493
F1 score: 0.9367537741250093


### Testing

In [11]:
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test])
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(300, dtype=float))

In [12]:
y_pred = clf.predict(X_test_vect_avg)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred, average='macro')}")

Accuracy: 0.9276613234689683
F1 score: 0.92489612550492


### Validation

In [13]:
test_df = pd.read_csv('raw_data/balancedtest.csv', header=None, names=['label', 'text'])
test_df['text_clean'] = test_df['text'].apply(lambda x: preprocess(x))
test_df['clean_tokens'] = test_df['text_clean'].apply(lambda x: tokenize(x))
X_val = test_df['clean_tokens'].values
y_val = test_df['label'].values

In [14]:
X_val_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_val])
X_val_vect_avg = []
for v in X_val_vect:
    if v.size:
        X_val_vect_avg.append(v.mean(axis=0))
    else:
        X_val_vect_avg.append(np.zeros(300, dtype=float))

In [15]:
y_pred_val = clf.predict(X_val_vect_avg)
print(f"Accuracy: {accuracy_score(y_val, y_pred_val)}")
print(f"F1 score: {f1_score(y_val, y_pred_val, average='macro')}")

Accuracy: 0.696
F1 score: 0.6860440146768954


## gloVE

In [16]:
embeddings_index = {}
f = open('raw_data/glove.840B.300d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
       coefs = np.asarray(values[1:], dtype='float32')
       embeddings_index[word] = coefs
    except ValueError:
       pass
f.close()

2196018it [01:53, 19431.55it/s]


In [17]:
def sent2vec(words):
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

### Testing

In [ ]:
xtrain_glove = [sent2vec(x) for x in tqdm(train_df['clean_tokens'])]
xtrain_glove = np.array(xtrain_glove)

 65%|██████▍   | 31541/48652 [00:23<00:20, 850.18it/s] 

In [ ]:
y = train_df['label']
clf = LogisticRegression(solver='sag').fit(xtrain_glove, y)
y_pred = clf.predict(xtrain_glove)
print(f"Accuracy: {accuracy_score(y, y_pred)}")
print(f"F1 score: {f1_score(y, y_pred, average='macro')}")

### Validation

In [ ]:
xtest_glove = [sent2vec(x) for x in tqdm(test_df['clean_tokens'])]
xtest_glove = np.array(xtest_glove)
y = test_df['label']
y_pred_val = clf.predict(xtest_glove)
print(f"Accuracy: {accuracy_score(y, y_pred_val)}")
print(f"F1 score: {f1_score(y, y_pred_val, average='macro')}")